# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import pprint

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read csv
weather_csv = pd.read_csv("../WeatherPy/output/weather_data.csv")

# Convert csv into data frame
weather_data = pd.DataFrame(weather_csv)

# Clean data frame 
weather_data=weather_data.drop(labels='Unnamed: 0', axis=1)

weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,WindSpeed,Country,Date
0,Nianzishan,47.52,122.88,56.05,46,4,4.00,CN,1589042647
1,Bluff,-46.60,168.33,52.68,80,30,10.18,NZ,1589042058
2,Ubari,26.59,12.78,96.85,9,0,11.03,LY,1589042149
3,Nowy Dwór Gdański,54.21,19.12,59.00,58,37,6.93,PL,1589042648
4,Hithadhoo,-0.60,73.08,85.14,79,100,13.87,MV,1589041968
...,...,...,...,...,...,...,...,...,...
564,Preeceville,51.95,-102.67,37.22,75,100,7.45,CA,1589042584
565,Whitehorse,60.72,-135.05,53.01,45,20,2.24,CA,1589042793
566,Pisco,-13.70,-76.22,71.01,82,30,8.05,PE,1589042793
567,Celestún,20.87,-90.40,88.32,58,86,4.94,MX,1589042794


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Set variables for location and humidity
locations_ = weather_data[['Lat', 'Lng']]
humidity = weather_data['Humidity'].astype(float)


In [4]:
# figure_layout = {
#     'width': '600px',
#     'height': '400px',
#     'border': '1px solid black',
#     'padding': '1px',
#     'margin': '0 auto 0 auto'
# } layout=figure_layout

# Create gmap with heat layer
fig = gmaps.figure(zoom_level = 1.2, center = (25, 25))
heat_layer = gmaps.heatmap_layer(locations_, weights = humidity, dissipating = False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Filter data to ideal vacation weather
vacation_data = weather_data.loc[(weather_data['Max Temp'] > 50) & (weather_data['Max Temp'] < 80) & (weather_data['Humidity'] < 35) & 
                 (weather_data['Cloudiness'] < 10) & (weather_data['WindSpeed'] < 10)] 

vacation_data = vacation_data.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store filtered data into data frame
hotel_df = pd.DataFrame(vacation_data)

# Add 'Hotel Name' column
hotel_df['Hotel Name'] = ''

# Clean data
hotel_df = hotel_df.reset_index().drop(labels='index', axis = 1)
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,WindSpeed,Country,Date,Hotel Name
0,Altay,47.87,88.12,55.72,33,0,9.26,CN,1589042572,
1,Taos,36.41,-105.57,57.99,26,1,8.05,US,1589042496,
2,Korla,41.76,86.15,68.95,23,9,3.76,CN,1589042139,
3,Yuty,-26.60,-56.25,78.17,30,0,8.97,PY,1589042684,
4,Saint George,37.10,-113.58,79.00,17,1,3.36,US,1589042685,
5,Gemeente Wageningen,51.97,5.67,75.20,27,0,5.82,NL,1589042686,
6,San Juan,-31.54,-68.54,77.38,23,1,4.36,AR,1589042713,
7,Tarata,-17.62,-66.02,75.20,22,0,1.86,BO,1589042727,
8,Springbok,-29.66,17.89,70.11,23,0,9.86,ZA,1589042731,
9,Neyshabur,36.21,58.80,71.67,23,0,4.09,IR,1589042744,


In [7]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# search parameters
params = {
    "radius": 5000,
    "type": 'lodging',
    "search": 'hotel',
    "key": g_key,
}

# iterate through rows to pull location for seach and add hotel name to data frame
for index, row in hotel_df.iterrows():
    
    locations = f"{row['Lat']} {row['Lng']}"
    params['location'] = locations
        
    response = requests.get(base_url, params=params).json()

    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        
    except IndexError:
        hotel_df.loc[index, 'Hotel Name'] = ''
        
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != '']
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,WindSpeed,Country,Date,Hotel Name
0,Altay,47.87,88.12,55.72,33,0,9.26,CN,1589042572,Jinqiao Hotel
1,Taos,36.41,-105.57,57.99,26,1,8.05,US,1589042496,Inn on La Loma Plaza
2,Korla,41.76,86.15,68.95,23,9,3.76,CN,1589042139,Gulisitan Hotel
3,Yuty,-26.60,-56.25,78.17,30,0,8.97,PY,1589042684,Hotel Independencia
4,Saint George,37.10,-113.58,79.00,17,1,3.36,US,1589042685,Best Western Plus Abbey Inn
5,Gemeente Wageningen,51.97,5.67,75.20,27,0,5.82,NL,1589042686,Fletcher Hotel-Restaurant De Wageningsche Berg
6,San Juan,-31.54,-68.54,77.38,23,1,4.36,AR,1589042713,Hotel Provincial
7,Tarata,-17.62,-66.02,75.20,22,0,1.86,BO,1589042727,TARA TARA Centro Alternativo Hosting
8,Springbok,-29.66,17.89,70.11,23,0,9.86,ZA,1589042731,Faithful Guest Lodge
9,Neyshabur,36.21,58.80,71.67,23,0,4.09,IR,1589042744,Amiran Hotel


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
locations


,Lat,Lng
0,47.87,88.12
1,36.41,-105.57
2,41.76,86.15
3,-26.60,-56.25
4,37.10,-113.58
5,51.97,5.67
6,-31.54,-68.54
7,-17.62,-66.02
8,-29.66,17.89
9,36.21,58.80


In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hotel_info)

fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))